In [8]:
import requests
from bs4 import BeautifulSoup
import time
import pandas as pd

# Continuação do script para extração dos ganhadores de óscar e nacionalidade
# Começar com a leitura e entendimento do que foi feito anteriormente
# Apenas fiz em outro programa para treinar mais uma vez toda a lógica e fixar conceitos de webscraping.

headers = {
    "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/555.36 (KHTML, like Gecko) Chrome/126.0.0.0 Safari/555.36"
}

def get_person_details(profile_url):
    person_details = {}
    if not profile_url:
        return person_details

    print(f"    Acessando perfil: {profile_url}")
    try:
        person_response = requests.get(profile_url, headers=headers)
        person_response.raise_for_status()
        person_soup = BeautifulSoup(person_response.text, 'html.parser')

        birth_place_label = person_soup.find('span', class_='ipc-metadata-list-item__label', string='Local de Nascimento')
        
        if birth_place_label:
            content_container = birth_place_label.find_next_sibling('div', class_='ipc-metadata-list-item__content-container')
            if content_container:
                birth_place_link = content_container.find('a', class_='ipc-link--base')
                if birth_place_link:
                    full_birth_place = birth_place_link.text.strip()
                    parts = [p.strip() for p in full_birth_place.split(',')]
                    nationalidade = parts[-1] if parts else None
                    person_details['Nacionalidade'] = nationalidade
                    print(f"      Encontrado Local de Nascimento: {full_birth_place}, Nacionalidade: {nationalidade}")
        else:
            birth_place_link_element = person_soup.find('a', class_='ipc-metadata-list-item__list-content-item', href=lambda href: href and 'birth_place' in href)
            if birth_place_link_element:
                full_birth_place = birth_place_link_element.text.strip()
                parts = [p.strip() for p in full_birth_place.split(',')]
                nationalidade = parts[-1] if parts else None
                person_details['Nacionalidade'] = nationalidade
                print(f"      Encontrado Local de Nascimento (alt): {full_birth_place}, Nacionalidade: {nationalidade}")

        person_details['Etnia'] = None # No site IMDb, etnia não faz parte do portfólio, infelizmente.

    except requests.exceptions.HTTPError as e:
        print(f"    Erro HTTP ao acessar o perfil {profile_url}: {e}")
    except requests.exceptions.RequestException as e:
        print(f"    Erro de requisição ao acessar o perfil {profile_url}: {e}")
    except Exception as e:
        print(f"    Ocorreu um erro inesperado ao processar o perfil {profile_url}: {e}")
    
    time.sleep(1.5) # Respeite o delay entre as requisições

    return person_details


def extrair_vencedores_oscar_2005_em_diante():
    ano_inicio = 2005
    ano_fim = 2025 # O ano final será incluído

    base_url = "https://www.imdb.com/pt/event/ev0000003/{ano}/1/?ref_=ev_acd_tl_yr_1"

    resultados_oscar = []

    # Dicionário atualizado com os novos IDs para 2005+
    categorias_interesse = {
        "Melhor Ator": [
            "BestActorInALeadingRole", "Melhor Ator Principal",
            "oscar_best_performance_by_an_actor_in_a_leading_role"
        ],
        "Melhor Atriz": [
            "BestActressInALeadingRole", "Melhor Atriz Principal",
            "oscar_best_performance_by_an_actress_in_a_leading_role"
        ],
        "Melhor Diretor": [
            "BestDirector", "Melhor Diretor",
            "oscar_best_achievement_in_directing"
        ],
        "Melhor Ator Coadjuvante": [
            "BestActorInASupportingRole", "Melhor Ator Coadjuvante",
            "oscar_best_performance_by_an_actor_in_a_supporting_role"
        ],
        "Melhor Atriz Coadjuvante": [
            "BestActressInASupportingRole", "Melhor Atriz Coadjuvante",
            "oscar_best_performance_by_an_actress_in_a_supporting_role"
        ]
    }

    for ano in range(ano_inicio, ano_fim + 1):
        url_ano = base_url.format(ano=ano)
        print(f"\nAcessando: {url_ano}")

        dados_ano = {"Ano": ano}
        encontrou_vencedores_no_ano = False

        try:
            response = requests.get(url_ano, headers=headers)
            response.raise_for_status()
            soup = BeautifulSoup(response.text, 'html.parser')

            # Tenta encontrar por IDs específicos primeiro (comum em anos mais antigos ou layouts específicos)
            for category_key, patterns in categorias_interesse.items():
                for pattern in patterns:
                    if pattern.startswith("oscar_"):
                        element_by_id = soup.find(id=pattern)
                        if element_by_id and category_key not in dados_ano: # Evita duplicar se já achou por outro padrão
                            ancestor = element_by_id.find_parent('div', class_='event-widgets__award-section')
                            if not ancestor:
                                ancestor = element_by_id.find_parent('section', class_='ipc-page-section')

                            winner_name = None
                            person_profile_url = None

                            if ancestor:
                                winner_signpost_div = ancestor.find('div', class_='ipc-signpost__text', string='Vencedor')
                                if winner_signpost_div:
                                    content_container = winner_signpost_div.find_parent('div', class_='ipc-metadata-list-item__content-container')
                                    if content_container:
                                        person_name_ul = content_container.find('ul', class_=lambda c: c and 'ipc-inline-list' in c.split())
                                        if person_name_ul:
                                            first_li = person_name_ul.find('li', class_='ipc-inline-list__item')
                                            if first_li:
                                                winner_name_element = first_li.find('a')
                                                
                                                if winner_name_element:
                                                    winner_name = winner_name_element.text.strip()
                                                    person_profile_url = "https://www.imdb.com" + winner_name_element['href']
                                                    
                                                    if winner_name:
                                                        dados_ano[category_key] = winner_name
                                                        encontrou_vencedores_no_ano = True
                                                        if person_profile_url:
                                                            person_data = get_person_details(person_profile_url)
                                                            dados_ano[f"{category_key}_Nacionalidade"] = person_data.get('Nacionalidade')
                                                            dados_ano[f"{category_key}_Etnia"] = person_data.get('Etnia')
                                                        else:
                                                            dados_ano[f"{category_key}_Nacionalidade"] = None
                                                            dados_ano[f"{category_key}_Etnia"] = None
                                                        print(f"  Encontrado {category_key} (via ID): {winner_name}")
                                                        break # Sai do loop de patterns se encontrar

            # Lógica principal de busca nas sections para os anos mais recentes e categorias sem IDs específicos
            award_sections = soup.find_all('section', class_='ipc-page-section')

            for section in award_sections:
                section_data_testid = section.get('data-testid', '')
                category_title_element = section.find('h3', class_='ipc-title__text--reduced')
                category_name_from_h3 = None
                if category_title_element:
                    category_name_from_h3 = category_title_element.find('span').text if category_title_element.find('span') else category_title_element.text.strip()

                # Ignora categorias de "Diretor Assistente" que podem causar conflito com "Melhor Diretor"
                if "assistant director" in section_data_testid.lower() or \
                   "diretor assistente" in (category_name_from_h3 or "").lower():
                    continue

                matched_category_key = None
                for key, patterns in categorias_interesse.items():
                    lower_patterns = [p.lower() for p in patterns]
                    section_id = section.get('id', '')
                    if (any(p.lower() in section_data_testid.lower() for p in lower_patterns) or
                        (category_name_from_h3 and any(p.lower() in (category_name_from_h3.lower() or "") for p in lower_patterns)) or
                        (section_id and any(p.lower() == section_id.lower() for p in lower_patterns))):
                        matched_category_key = key
                        break
                
                if matched_category_key:
                    if matched_category_key in dados_ano: # Se já encontramos para este ano/categoria (via ID), pulamos
                        continue

                    winner_signpost_div = section.find('div', class_='ipc-signpost__text', string='Vencedor')

                    if winner_signpost_div:
                        winner_name = None
                        person_profile_url = None

                        ancestor = winner_signpost_div.find_parent('div', class_='ipc-metadata-list-item__content-container')
                        if ancestor:
                            person_name_ul = ancestor.find('ul', class_=lambda c: c and 'ipc-inline-list' in c.split())
                            if person_name_ul:
                                first_li = person_name_ul.find('li', class_='ipc-inline-list__item')
                                if first_li:
                                    winner_name_element = first_li.find('a')
                                    if winner_name_element:
                                        winner_name = winner_name_element.text.strip()
                                        person_profile_url = "https://www.imdb.com" + winner_name_element['href']
                                
                        if winner_name:
                            dados_ano[matched_category_key] = winner_name
                            encontrou_vencedores_no_ano = True

                            if person_profile_url:
                                person_data = get_person_details(person_profile_url)
                                dados_ano[f"{matched_category_key}_Nacionalidade"] = person_data.get('Nacionalidade')
                                dados_ano[f"{matched_category_key}_Etnia"] = person_data.get('Etnia')
                                print(f"    Nacionalidade: {dados_ano.get(f'{matched_category_key}_Nacionalidade', 'N/A')}")
                            else:
                                dados_ano[f"{matched_category_key}_Nacionalidade"] = None
                                dados_ano[f"{matched_category_key}_Etnia"] = None

                            print(f"  Encontrado {matched_category_key}: {winner_name}")
                        else:
                            print(f"  Não encontrado nome do vencedor para {matched_category_key} no ano {ano}.")
                    else:
                        print(f"  Não encontrado 'Vencedor' para {matched_category_key} no ano {ano}.")

            if encontrou_vencedores_no_ano:
                resultados_oscar.append(dados_ano)
            else:
                if response.status_code != 404:
                    print(f"  Nenhum vencedor das categorias de interesse encontrado para o ano {ano}.")

        except requests.exceptions.HTTPError as e:
            print(f"  Erro HTTP ao acessar a URL {url_ano}: {e}")
        except requests.exceptions.RequestException as e:
            print(f"  Erro de requisição ao acessar a URL {url_ano}: {e}")
        except Exception as e:
            print(f"  Ocorreu um erro inesperado para o ano {ano}: {e}")

        time.sleep(2.5) # Respeite o delay entre as requisições

    return resultados_oscar

if __name__ == "__main__":
    dados_completos_oscar_2005_em_diante = extrair_vencedores_oscar_2005_em_diante()

    if dados_completos_oscar_2005_em_diante:
        df_oscar = pd.DataFrame(dados_completos_oscar_2005_em_diante)
        
        categorias_principais = [
            "Melhor Ator",
            "Melhor Atriz",
            "Melhor Diretor",
            "Melhor Ator Coadjuvante",
            "Melhor Atriz Coadjuvante"
        ]
        
        colunas_finais = ['Ano']
        for cat in categorias_principais:
            colunas_finais.append(cat)
            colunas_finais.append(f"{cat}_Nacionalidade")
            colunas_finais.append(f"{cat}_Etnia")

        for col in colunas_finais:
            if col not in df_oscar.columns:
                df_oscar[col] = None
        
        df_oscar = df_oscar[colunas_finais]

        print("\n--- Primeiras 25 linhas dos dados coletados (2005 em diante) ---")
        print(df_oscar.head(25).to_string())
        df_oscar.to_csv("ganhadores_oscar_pessoas_2005_em_diante.csv", index=False)
        print("\nDados salvos em 'ganhadores_oscar_pessoas_2005_em_diante.csv'")
        print(f"Total de anos com dados coletados: {len(dados_completos_oscar_2005_em_diante)}")
    else:
        print("\nNenhum dado do Oscar foi coletado a partir de 2005. Verifique os logs de erro acima.")


Acessando: https://www.imdb.com/pt/event/ev0000003/2005/1/?ref_=ev_acd_tl_yr_1
    Acessando perfil: https://www.imdb.com/pt/name/nm0004937/?ref_=ev_nom
      Encontrado Local de Nascimento (alt): Terrell, Texas, EUA, Nacionalidade: EUA
  Encontrado Melhor Ator (via ID): Jamie Foxx
    Acessando perfil: https://www.imdb.com/pt/name/nm0005476/?ref_=ev_nom
      Encontrado Local de Nascimento (alt): Lincoln, Nebraska, EUA, Nacionalidade: EUA
  Encontrado Melhor Atriz (via ID): Hilary Swank
    Acessando perfil: https://www.imdb.com/pt/name/nm0000142/?ref_=ev_nom
      Encontrado Local de Nascimento (alt): San Francisco, Califórnia, EUA, Nacionalidade: EUA
  Encontrado Melhor Diretor (via ID): Clint Eastwood
    Acessando perfil: https://www.imdb.com/pt/name/nm0000151/?ref_=ev_nom
      Encontrado Local de Nascimento (alt): Memphis, Tennessee, EUA, Nacionalidade: EUA
  Encontrado Melhor Ator Coadjuvante (via ID): Morgan Freeman
    Acessando perfil: https://www.imdb.com/pt/name/nm0000949